In [ ]:
import tensorflow as tf
import cvxpy as cp
import numpy as np


In [ ]:
#load data and preprocess data
mnist = tf.keras.datasets.mnist
#choose pictures of number 2 and number 6
(train_images, train_labels), (test_images,test_labels) =mnist.load_data()#include all numbers from 0 to 9 index_train=p.where((train labels ==2) | (train labels ==6))#index of numbers 3 and 5 in training data index_test=p.where((test_labels ==2) | (test_labels ==6))#index of numbers 3 and 5 in test data train images 26=train images [index train]
index_train=np.where((train_labels == 2) | (train_labels == 6))#index of numbers 3 and 5 in training data
index_test=np.where((test_labels ==2) | (test_labels ==6))#index of numbers 3 and 5 in test data
train_images_26=train_images[index_train]
train_images_26=train_images_26.reshape ((len (train_images_26), train_images_26[1].size)) #label of number 2: -1; label of number 6: +1 train_labels_26=train_labels[index_train].astype('int')
#label of number 2: -1; label of number 6: +1 
train_labels_26=train_labels[index_train].astype('int')
test_images_26=test_images[index_test]
test_images_26=test_images_26.reshape((len(test_images_26), train_images_26[1].size)) 
test_labels_26=test_labels[index_test].astype('int')
#change labels from '2' and '6' to '-1' and '+1' 
train_labels_26[np.where(train_labels_26==2)]= -1
train_labels_26[np.where(train_labels_26==6)]=1
test_labels_26[np.where(test_labels_26==2)]= -1
test_labels_26[np.where(test_labels_26==6)]=1
#Normalize 
train_images_26 = train_images_26/255
test_images_26 = test_images_26/255

In [ ]:
#task 1:Optimal Margin Classifier 
X1 = train_images_26[:300]
Y1 = train_labels_26[:300]
X1_test = test_images_26
Y1_test = test_labels_26
#creating W 1 vector
W = cp.Variable(784)#784x1
b = cp.Variable()#Scalar
const_equ1 = cp.multiply(Y1,((X1 @ W)+b))
#CVXPY constrains for the objective
constraints = [const_equ1 >=1]
#Objective Function
obj1 = cp.Minimize(cp.norm(W)**2)
#Problem Function
prob1 = cp.Problem(obj1,constraints)
prob1.solve()
#print("Staus", prob1.status)
#print("Optimal Value",(prob1.value))
#print("Optimal W* = \n",(W.value)," b* = \n",b.value)
optimal_W = W.value
optimal_b = b.value
#Prediction
predictedtrain_y = np.sign(np.dot(X1,optimal_W)+optimal_b)
training_error = np.mean([predictedtrain_y] != Y1)
print("Training Error(%):",training_error*100)
predictedtest_y = np.sign(np.dot(X1_test,optimal_W)+optimal_b)
test_error= np.mean([predictedtest_y] != Y1_test)
print("Testing Error(%):",test_error*100)

Training Error(%): 0.0
Testing Error(%): 3.4170854271356785


In [176]:
from tensorflow.python.ops.gen_array_ops import Shape
from numpy.core.multiarray import where
#task 2: Dual Form of Optimal Margin Classifier
alpha = cp.Variable(300)
const_equ2 = alpha.T @ Y1
Mij = X1 @ X1.T
beta = cp.multiply(alpha, Y1)
#CVXPY constrains for the objective
constraints = [const_equ2 == 0, alpha >= 0]
#Objective Function
obj2 = cp.Maximize(cp.sum(alpha)-((beta).T @ Mij @ beta)*0.5)
prob2 = cp.Problem(obj2,constraints)
prob2.solve()
#a*
optimal_alpha = alpha.value
#w*
optimal_W2_intermediate = np.multiply(Y1, optimal_alpha)
optimal_W2 = optimal_W2_intermediate @ X1
#Slice X according to Y labels -1 and 1
index_neg = []
index_pos = []
for i in range(0,299):
    if Y1[i] == -1:
      index_neg.append(i)
    else:
      index_pos.append(i)
X2 = np.zeros((149,784), dtype=float)
X3 = np.zeros((151,784), dtype=float)
X2 = X1[index_neg]
X3 = X1[index_pos]
A= np.dot(X2,optimal_W2.T)
B = np.dot(X3,optimal_W2.T)
optimal_b2 = (np.max(A)+np.min(B))*(-0.5)
#Prediction
predictedtrain_y2 = np.sign(np.dot(X1,optimal_W2)+optimal_b2)
training_error2 = np.mean([predictedtrain_y2] != Y1)
print("Training Error(%):",training_error2*100)
predictedtest_y2 = np.sign(np.dot(X1_test,optimal_W2)+optimal_b2)
test_error2= np.mean([predictedtest_y2] != Y1_test)
print("Testing Error(%):",test_error2*100)



Training Error(%): 0.0
Testing Error(%): 6.0


In [183]:
#task 3: Dual Form of Optimal Margin Classifier with Kernel Trick
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import pairwise_kernels
X1 = train_images_26[:300]
Y1 = train_labels_26[:300]
X1_test = test_images_26[:300]
Y1_test = test_labels_26[:300]
K_train=rbf_kernel(X1,gamma=1)
K_train_test=rbf_kernel(X1,X1_test,gamma=1)


alpha = cp.Variable(300)
const_equ3 = alpha.T @ Y1
#Kernel inner product
beta = cp.multiply(alpha, Y1)
#CVXPY constrains for the objective
constraints = [const_equ2 == 0, alpha >= 0]
#Objective Function
obj3 = cp.Maximize(cp.sum(alpha)-(beta).T @ K_train @ beta)
prob3 = cp.Problem(obj3,constraints)
prob3.solve()

#a*
optimal_alpha2 = alpha.value

#Slice X according to Y labels -1 and 1
index_neg = []
index_pos = []
for i in range(0,299):
    if Y1[i] == -1:
      index_neg.append(i)
    else:
      index_pos.append(i)
X4 = np.zeros((149,784), dtype=float)
X5 = np.zeros((151,784), dtype=float)
X4 = K_train[index_neg]
X5 = K_train[index_pos]

#find b*
C= X4 @ np.multiply(Y1, optimal_alpha2) 
D = X5 @ np.multiply(Y1, optimal_alpha2) 
E =  np.multiply(Y1, optimal_alpha2) @ K_train
F = np.multiply(Y1_test, optimal_alpha2) @ K_train_test
optimal_b2 = (np.max(C)+np.min(D))*(-0.5)

#Prediction
predictedtrain_y3 = np.sign(E+optimal_b2)
training_error3 = np.mean([predictedtrain_y3] != Y1)
print("Training Error(%):",training_error3*100)
predictedtest_y3 = np.sign(F + optimal_b2)
test_error3 = np.mean([predictedtest_y3] != Y1_test)
print("Testing Error(%):",test_error3*100)

Training Error(%): 0.0
Testing Error(%): 57.99999999999999


In [180]:
#Task 6: Knn using kernel fucntion
k2acc = 0
k2accrate = 0
testk2acc = 0
testk2accrate = 0
difference = []
for row in range(0,299):
    sourcepoint = K_train[row,:]
    for check in range(0,299):
            dist = np.sqrt(np.sum(np.square(np.subtract(sourcepoint,K_train[check,:]))))
            difference.append(dist)
    mindindex1 = difference.index(min(d for d in difference if d>0))
    difference[mindindex1] = 0
    mindindex2 =  difference.index(min(d for d in difference if d>0))
    difference[mindindex2] = 0
    mindindex3 =  difference.index(min(d for d in difference if d>0))
    dec = [Y1[mindindex1], Y1[mindindex2], Y1[mindindex3]]
    predictedY = max(dec, key=dec.count)
    #print("Predicted Y", predictedY, "Real Y", train_labels_26_w_dummy[row])
    if(predictedY == Y1[row]):
        k2acc = k2acc +1
    difference = []
    k2accrate = k2acc/10

print("K(k=3) means on Training Data using Kernel(%):",k2accrate)

for row in range(0,299):
    sourcepoint = K_train_test[row,:]
    for check in range(0,299):
            dist = np.sqrt(np.sum(np.square(np.subtract(sourcepoint,K_train_test[check,:]))))
            difference.append(dist)
    mindindex1 = difference.index(min(d for d in difference if d>0))
    difference[mindindex1] = 0
    mindindex2 =  difference.index(min(d for d in difference if d>0))
    difference[mindindex2] = 0
    mindindex3 =  difference.index(min(d for d in difference if d>0))
    dec = [Y1_test[mindindex1], Y1_test[mindindex2], Y1_test[mindindex3]]
    predictedY = max(dec, key=dec.count)
    #print("Predicted Y", predictedY, "Real Y", train_labels_26_w_dummy[row])
    if(predictedY == Y1_test[row]):
        testk2acc = testk2acc +1
    difference = []
    testk2accrate = (testk2acc/199)*10

print("K(k=3) means on Testing Data using Kernel(%):",testk2accrate)

# ######################for k = 5

k3acc = 0
k3accrate = 0
testk3acc = 0
testk3accrate = 0


for row in range(0,299):
    sourcepoint = K_train[row,:]
    for check in range(0,299):
            dist = np.sqrt(np.sum(np.square(np.subtract(sourcepoint,K_train[check,:]))))
            difference.append(dist)
    mindindex1 = difference.index(min(d for d in difference if d>0))
    difference[mindindex1] = 0
    mindindex2 =  difference.index(min(d for d in difference if d>0))
    difference[mindindex2] = 0
    mindindex3 =  difference.index(min(d for d in difference if d>0))
    difference[mindindex3] = 0
    mindindex4 =  difference.index(min(d for d in difference if d>0))
    difference[mindindex4] = 0
    mindindex5 =  difference.index(min(d for d in difference if d>0))
    dec = [Y1[mindindex1], Y1[mindindex2], Y1[mindindex3]]
    predictedY = max(dec, key=dec.count)
    #print("Predicted Y", predictedY, "Real Y", train_labels_26_w_dummy[row])
    if(predictedY == Y1[row]):
        k3acc = k3acc +1
    difference = []
    k3accrate = k3acc/10

print("K(k=5) means on Training Data using Kernel(%):",k3accrate)

for row in range(0,299):
    sourcepoint = K_train_test[row,:]
    for check in range(0,299):
            dist = np.sqrt(np.sum(np.square(np.subtract(sourcepoint,K_train_test[check,:]))))
            difference.append(dist)
    mindindex1 = difference.index(min(d for d in difference if d>0))
    difference[mindindex1] = 0
    mindindex2 =  difference.index(min(d for d in difference if d>0))
    difference[mindindex2] = 0
    mindindex3 =  difference.index(min(d for d in difference if d>0))
    difference[mindindex3] = 0
    mindindex4 =  difference.index(min(d for d in difference if d>0))
    difference[mindindex4] = 0
    mindindex5 =  difference.index(min(d for d in difference if d>0))
    dec = [Y1_test[mindindex1], Y1_test[mindindex2], Y1_test[mindindex3],
            Y1_test[mindindex4],Y1_test[mindindex5]]
    predictedY = max(dec, key=dec.count)
    #print("Predicted Y", predictedY, "Real Y", train_labels_26_w_dummy[row])
    if(predictedY == Y1_test[row]):
        testk3acc = testk3acc +1
    difference = []
    testk3accrate = (k3acc/199)*10
print("K(k=5) means on Testing Data using Kernel(%):",testk3accrate)

print("Comparison for all models in written part")



K(k=3) means on Training Data using Kernel(%): 27.4
K(k=3) means on Testing Data using Kernel(%): 7.386934673366834
K(k=5) means on Training Data using Kernel(%): 27.4
K(k=5) means on Testing Data using Kernel(%): 13.768844221105528
Comparison for all models in written part


K(k=3) means on Training Data without Kernel(%): 29.3
K(k=3) means on Testing Data(%): 14.824120603015077
K(k=5) means on Training Data without Kernel(%): 29.3
K(k=5) means on Testing Data(%): 14.723618090452263
Comparison for all models in written part
